### Tuning Parameters

In [ ]:
# Define HOG parameters
p_HOG = {'orient':9, 'pix_per_cell':8, 'cell_per_block':2}

ystart = 400
ystop = 656
scale = 1.5

### Library Import and Data Read

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline

from skimage.feature import hog
from skimage import color, exposure
from sklearn.cross_validation import train_test_split
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition
from scipy.ndimage.measurements import label

from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
# cars = glob.glob('./DATASETS/vehicles/*/*.png')
# notcars = glob.glob('./DATASETS/non-vehicles/*/*.png')

In [ ]:
cars = glob.glob('./DATASETS/vehicles_smallset/*/*.jpeg')
notcars = glob.glob('./DATASETS/non-vehicles_smallset/*/*.jpeg')

### Feature Extraction

In [ ]:
# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    return features

# Define a function to compute color histogram features  
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return features

# def my_hog():

In [ ]:
def extract_features(img_name):
    img = cv2.imread(img_name)
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features_hog = hog(img_gray,
                       orientations=p_HOG['orient'],
                       pixels_per_cell=(p_HOG['pix_per_cell'], p_HOG['pix_per_cell']),
                       cells_per_block=(p_HOG['cell_per_block'], p_HOG['cell_per_block']),
                       visualise=False, feature_vector=True)
#     features_spatial = bin_spatial(img)
    features_color_hist = color_hist(img)
#     print(" ")
#     print(np.max(features_hog))
#     print(np.max(features_spatial))
#     print(np.max(features_color_hist))
    print('features_hog ', features_hog.shape )
#     print('features_spatial ',features_spatial.shape)
    print('features_color_hist ', features_color_hist.shape)
    
#     return (np.concatenate((features_hog, features_spatial, features_color_hist)))
    return (np.concatenate((features_hog,features_color_hist)))
#     return (np.concatenate((features_hog, features_spatial)))

In [ ]:
features_list = []

for car in cars:
    features = extract_features(car)
    features_list.append(features)
    
for notcar in notcars:
    features = extract_features(notcar)
    features_list.append(features)
    
features_array = np.asarray(features_list)
features_array.shape

### Feature Normalization


In [ ]:
## why not vstack?
# y = np.vstack((np.ones((5,1)), np.zeros((4,1))))

In [ ]:
y_data = np.hstack((np.ones(len(cars)), np.zeros(len(notcars))))

In [ ]:
print(y_data.shape)

In [ ]:
def scale_features(features):
    # Fit a per-column scaler
    XScaler = StandardScaler().fit(features)
    # Apply the scaler to X
    return XScaler.transform(features)

In [ ]:
XScaler = StandardScaler().fit(features_array)
X_data = XScaler.transform(features_array)

In [ ]:
print(XScaler)

In [ ]:
X_data.shape

In [ ]:
# PCA = decomposition.PCA(n_components=500)
# PCA.fit(X_data)
# X_data = PCA.transform(X_data)
# X_data.shape

In [ ]:
# plt.figure()
# plt.boxplot(features_array, 1, '')

In [ ]:
# plt.figure()
# plt.boxplot(X_data, 1, '')

In [ ]:
if len(features_array) > 0:
    car_ind = np.random.randint(0, len(cars))
    # Plot an example of raw and scaled features
    fig = plt.figure(figsize=(12,4))
    plt.subplot(131)
    plt.imshow(mpimg.imread(cars[car_ind]))
    plt.title('Original Image')
    plt.subplot(132)
    plt.plot(features_array[car_ind])
    plt.title('Raw Features')
    plt.subplot(133)
    plt.plot(X_data[car_ind])
    plt.title('Normalized Features')
    fig.tight_layout()
else: 
    print('Your function only returns empty feature vectors...')

### SVM or Decision Tree or Neural Networks

In [ ]:
# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size=0.2, random_state=rand_state)

In [ ]:
svc = LinearSVC()
svc.fit(X_train, y_train)

In [ ]:
print('Test Accuracy of SVC = ', svc.score(X_test, y_test))

## Test Phase

In [ ]:
# Define a single function that can extract features using hog sub-sampling and make predictions
def find_cars(img, ystart, ystop, scale, svc,  p_HOG):
    draw_img = np.copy(img)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    ## bounding boxes
    bboxes=[]
#     img = img.astype(np.float32)/255
    img_roi = img[ystart:ystop,:,:]
    if scale != 1:
        imshape = img_roi.shape
        img_roi = cv2.resize(img_roi, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
        
    ch1 = img_roi[:,:,0]
    ch2 = img_roi[:,:,1]
    ch3 = img_roi[:,:,2]

    # Define blocks and steps as above
    # ch1.shape[1] = 1280
    # nxblocks = 159     
    nxblocks = (ch1.shape[1] // p_HOG['pix_per_cell'])-1
    # ch1.shape[0] = 720
    # nyblocks = 89
    nyblocks = (ch1.shape[0] // p_HOG['pix_per_cell'])-1
    # number of features per block = (2**2) * 9 = 36
    nfeat_per_block = (p_HOG['cell_per_block']**2) * p_HOG['orient']
    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    # nblocks_per_window = 7
    nblocks_per_window = (window // p_HOG['pix_per_cell'])-1
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    # nxsteps = 76
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
    # nysteps = 41
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step
    
    # Compute individual channel HOG features for the entire image
    features_hog_frame = hog(gray,
                             orientations=p_HOG['orient'],
                             pixels_per_cell=(p_HOG['pix_per_cell'],p_HOG['pix_per_cell']),
                             cells_per_block=(p_HOG['cell_per_block'],p_HOG['cell_per_block']),
                             visualise=False, feature_vector=False)
    
    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
            # Extract HOG for this patch
#             print('features_hog_frame[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window]')
#             print(ypos, ypos+nblocks_per_window, xpos, xpos+nblocks_per_window)
            features_hog = features_hog_frame[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
#             print('features_hog')
#             print(features_hog.shape)
            xleft = xpos*p_HOG['pix_per_cell']
            ytop = ypos*p_HOG['pix_per_cell']

            # Extract the image patch
            img_patch = cv2.resize(img_roi[ytop:ytop+window, xleft:xleft+window], (64,64))
          
            # Get color features
#             features_spatial = bin_spatial(img_patch)
#             print('features_spatial ',features_spatial.shape)
            
            features_color_hist = color_hist(img_patch)
#             print('features_color_hist', features_color_hist.shape)
            
#             print("Testing: ")
#             print(np.max(features_hog))
#             print(np.max(features_spatial))
#             print(np.max(features_color_hist))

            # Scale features and make a prediction
#             test_features = np.hstack((spatial_features, hist_features, features_hog)).reshape(1, -1)
#             test_features = np.concatenate((features_hog, features_spatial, features_color_hist)).reshape(1, -1)
#             test_features = np.concatenate((features_hog, features_spatial)).reshape(1, -1)
            test_features = np.concatenate((features_hog, features_color_hist )).reshape(1, -1)
#             test_features = scale_features(test_features)
            test_features = XScaler.transform(test_features)
#             test_features = PCA.transform(test_features)
            test_features.shape
        
            #test_features = X_scaler.transform(np.hstack((shape_feat, hist_feat)).reshape(1, -1))    
            test_prediction = svc.predict(test_features)
            
            
            if test_prediction == 1:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                x1 = xbox_left
                y1 = ystart+ytop_draw
                x2 = xbox_left+win_draw
                y2 = ystart+ytop_draw+win_draw
                bboxes.append(((x1, y1),(x2,y2)))
#                 print("bboxes", bboxes)
                cv2.rectangle(draw_img,(x1, y1),(x2,y2),(0,0,255),6) 
                
    return draw_img, bboxes

def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap# Iterate through list of bboxes
    
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img

In [ ]:
# img = cv2.imread('./test_images/test1.jpg')
# img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 

In [ ]:
def process_image(img):
#     img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    out_img, bbox_list = find_cars(img, ystart, ystop, scale, svc, p_HOG)
    heat = np.zeros_like(out_img[:,:,0]).astype(np.float)

    # Add heat to each box in box list
    heat = add_heat(heat,bbox_list)

    # Apply threshold to help remove false positives
    heat = apply_threshold(heat,0.3)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)

    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    draw_img = draw_labeled_bboxes(np.copy(img), labels)
    return draw_img

In [ ]:
clip1 = VideoFileClip("./project_video.mp4")
vehicle_clip = clip1.fl_image(process_image)
img = clip1.get_frame(9999999)

out_img, bbox_list = find_cars(img, ystart, ystop, scale, svc, p_HOG)

heat = np.zeros_like(out_img[:,:,0]).astype(np.float)

# Add heat to each box in box list
heat = add_heat(heat,bbox_list)
    
# Apply threshold to help remove false positives
heat = apply_threshold(heat,0.3)

# Visualize the heatmap when displaying    
heatmap = np.clip(heat, 0, 255)

# Find final boxes from heatmap using label function
labels = label(heatmap)
draw_img = draw_labeled_bboxes(np.copy(img), labels)

fig = plt.figure()
plt.subplot(121)
plt.imshow(draw_img)
plt.title('Car Positions')
plt.subplot(122)
plt.imshow(heatmap, cmap='gray')
plt.title('Heat Map')
fig.tight_layout()

In [ ]:
vehicle_output = './vehicle_output.mp4'
%time vehicle_clip.write_videofile(vehicle_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(vehicle_output))